# Background Removal with OpenCV

This notebook demonstrates how to remove backgrounds from images using OpenCV's GrabCut algorithm.

## Features
- Batch processing of multiple images
- Automatic background removal using GrabCut
- Transparent PNG output
- Progress tracking
- Error handling and logging


In [ ]:
# Import required libraries
import cv2
import numpy as np
import os
from pathlib import Path
import logging
from tqdm import tqdm
import matplotlib.pyplot as plt

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')
logger = logging.getLogger(__name__)


In [ ]:
# Define input and output directories
input_dir = Path("images")
output_dir = Path("output")

# Validate folder structure
if not input_dir.exists():
    logger.error("Input folder 'images/' not found.")
    raise FileNotFoundError("Missing input folder.")
    
output_dir.mkdir(exist_ok=True)
logger.info(f"Output directory created: {output_dir}")

# Allowed image extensions
valid_extensions = {".jpg", ".jpeg", ".png", ".bmp", ".tiff"}


In [ ]:
def process_image(image_path, output_dir, margin=0.05):
    """
    Process a single image to remove background using GrabCut algorithm.
    
    Args:
        image_path (Path): Path to input image
        output_dir (Path): Output directory
        margin (float): Margin percentage for GrabCut rectangle
    
    Returns:
        bool: Success status
    """
    try:
        # Read image
        img = cv2.imread(str(image_path))
        if img is None:
            logger.warning(f"Skipping {image_path.name} (unable to read)")
            return False
            
        # Get image dimensions
        height, width = img.shape[:2]
        
        # Calculate rectangle for GrabCut (with margin)
        x = int(width * margin)
        y = int(height * margin)
        rect = (x, y, width - 2*x, height - 2*y)
        
        # Create mask and models for GrabCut
        mask = np.zeros(img.shape[:2], np.uint8)
        bgd_model = np.zeros((1, 65), np.float64)
        fgd_model = np.zeros((1, 65), np.float64)
        
        # Apply GrabCut algorithm
        cv2.grabCut(img, mask, rect, bgd_model, fgd_model, 5, cv2.GC_INIT_WITH_RECT)
        
        # Create binary mask
        mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype("uint8")
        
        # Convert to RGBA and apply mask to alpha channel
        output_rgba = cv2.cvtColor(img, cv2.COLOR_BGR2BGRA)
        output_rgba[:, :, 3] = mask2 * 255  # 0 for background, 255 for foreground
        
        # Save as PNG to preserve transparency
        output_path = output_dir / image_path.with_suffix('.png').name
        cv2.imwrite(str(output_path), output_rgba)
        logger.info(f"✓ Saved: {output_path.name}")
        return True
        
    except Exception as e:
        logger.error(f"✗ Error processing {image_path.name}: {e}")
        return False


In [ ]:
# Find all valid image files
image_files = [f for f in input_dir.iterdir() if f.suffix.lower() in valid_extensions]

if not image_files:
    logger.warning("No valid image files found in 'images/' folder.")
    logger.info(f"Please add images with extensions: {', '.join(valid_extensions)}")
else:
    logger.info(f"Found {len(image_files)} image(s) to process")
    
    # Process all images with progress bar
    successful = 0
    for image_path in tqdm(image_files, desc="Processing images", unit="image"):
        if process_image(image_path, output_dir):
            successful += 1
    
    logger.info(f"Processing complete! {successful}/{len(image_files)} images processed successfully.")


## Visualization

Let's visualize the results by comparing original and processed images:


In [ ]:
def display_results(original_path, processed_path, max_images=3):
    """
    Display original and processed images side by side.
    """
    processed_files = list(output_dir.glob("*.png"))
    
    if not processed_files:
        print("No processed images found.")
        return
    
    # Display up to max_images
    for i, processed_file in enumerate(processed_files[:max_images]):
        # Find corresponding original
        original_file = input_dir / processed_file.name.replace('.png', '')
        
        # Try different extensions
        for ext in valid_extensions:
            potential_original = original_file.with_suffix(ext)
            if potential_original.exists():
                original_file = potential_original
                break
        
        if original_file.exists():
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
            
            # Original image
            original_img = cv2.imread(str(original_file))
            original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
            ax1.imshow(original_img)
            ax1.set_title(f"Original: {original_file.name}")
            ax1.axis('off')
            
            # Processed image
            processed_img = cv2.imread(str(processed_file), cv2.IMREAD_UNCHANGED)
            if processed_img.shape[2] == 4:  # RGBA
                # Create white background for display
                white_bg = np.ones((processed_img.shape[0], processed_img.shape[1], 3), dtype=np.uint8) * 255
                alpha = processed_img[:, :, 3:4] / 255.0
                rgb = processed_img[:, :, :3]
                processed_img = (alpha * rgb + (1 - alpha) * white_bg).astype(np.uint8)
            
            ax2.imshow(processed_img)
            ax2.set_title(f"Processed: {processed_file.name}")
            ax2.axis('off')
            
            plt.tight_layout()
            plt.show()
        else:
            print(f"Could not find original for {processed_file.name}")

# Display results if any images were processed
if 'image_files' in locals() and image_files:
    display_results(input_dir, output_dir)


## Tips for Better Results

1. **Image Quality**: Use high-quality images with clear contrast between subject and background
2. **Subject Position**: Center your subject in the image for best results
3. **Background Complexity**: Simple, uniform backgrounds work better than complex patterns
4. **Lighting**: Well-lit subjects with distinct shadows work best
5. **Margin Adjustment**: You can adjust the margin parameter (default 5%) if needed

## Limitations

- May struggle with complex backgrounds or subjects that blend into the background
- No semantic understanding of objects
- Best suited for images with clear foreground/background separation

## Next Steps

For more advanced background removal, consider:
- Deep learning models like U-2-Net, MODNet, or MediaPipe Selfie Segmentation
- Manual refinement tools
- Different segmentation algorithms
